In [1]:
import pandas as pd
import os

In [2]:
## get unique samples to build references
meta = pd.read_csv('real_data_coverage.csv')

In [3]:
unique_samples = meta['cell_line'].unique()

In [4]:
unique_df = pd.DataFrame(unique_samples, columns=['unique_samples'])
unique_df.to_csv('unique_samples.txt', header=None, index=False, sep = '\t')

In [5]:
### read populations and write samples in real data

In [6]:
pop_info = pd.read_table('1000G_2504_high_coverage.sequence.index', comment ='#', header = None)

In [7]:
header = '#ENA_FILE_PATH	MD5SUM	RUN_ID	STUDY_ID	STUDY_NAME	CENTER_NAME	SUBMISSION_ID	SUBMISSION_DATE	SAMPLE_ID	SAMPLE_NAME	POPULATION	EXPERIMENT_ID	INSTRUMENT_PLATFORM	INSTRUMENT_MODEL	LIBRARY_NAME	RUN_NAME	INSERT_SIZE	LIBRARY_LAYOUT	PAIRED_FASTQ	READ_COUNT	BASE_COUNT	ANALYSIS_GROUP'.split()

In [8]:
pop_info.columns = header

In [9]:
## write all 2504 samples
pop_all_samples = pop_info['LIBRARY_NAME']

In [10]:
pop_all_samples.to_csv('2504_samples.txt', header=None, index=False, sep = '\t')

In [11]:
african_populations = [
    'YRI',  # Yoruba in Nigeria
    'LWK',  # Luhya in Webuye, Kenya
    'GWD',  # Gambian in Western Divisions in the Gambia
    'MSL',  # Mende in Sierra Leone
    'ESN',  # Esan in Nigeria
    'ASW',  # Americans of African Ancestry in SW USA
    'ACB'   # African Caribbean in Barbados
]


european_populations = [
    'CEU',  # Utah Residents (CEPH) with Northern and Western European Ancestry
    'TSI',  # Toscani in Italy
    'FIN',  # Finnish in Finland
    'GBR',  # British in England and Scotland
    'IBS'   # Iberian populations in Spain
]

In [12]:
pick = pop_info.POPULATION.isin(european_populations)
eur = pop_info['LIBRARY_NAME'][pick]
eur_df = unique_df[unique_df.unique_samples.isin(eur)]
eur_df.to_csv('eur_samples.txt', header=None, index=False, sep = '\t')
len(eur_df)

58

In [14]:
pick = pop_info.POPULATION.isin(african_populations)
afr = pop_info['LIBRARY_NAME'][pick]
afr_df = unique_df[unique_df.unique_samples.isin(afr)]
afr_df.to_csv('afr_samples.txt', header=None, index=False, sep = '\t')

In [15]:
len(afr_df)

60

In [16]:
data = pd.read_table('meta_experiment_A.txt', header = None, sep='\s+')

In [17]:
data.columns = ['date', 'size', 'unit', 'file']

In [18]:
data['cell_line'] = [x.split('-')[0] for x in data['file']]
data['coverage'] = [x.split('-')[1] for x in data['file']]
data['replicate'] = [x.split('-')[2] for x in data['file']]

In [23]:
type(eur)

pandas.core.series.Series

In [59]:
def get_meta_data(data, pop, relicate):
    path_prefix = 'https://gencove-sbir.s3.amazonaws.com/fastq/experiment-A/'
    pick = (data['cell_line'].isin(pop)) & (data['replicate'] == str(relicate))
    tem = data[pick]
    sample_id = []
    read1 = []
    read2 = []
    for file in tem['file']:
        if 'R1' in file:
            sample_id.append(file.split('-')[0])
            read1.append(path_prefix + file)
        else:
            read2.append(path_prefix + file)
    return pd.DataFrame({'sample_id': sample_id, 'read1': read1, 'read2': read2})

In [61]:
eur_rep1 = get_meta_data(data, eur, '0')

In [63]:
eur_rep1.to_csv('eur_rep1.csv', index=None)